In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from talib import abstract
import math

plt.style.use('fivethirtyeight')

In [3]:
# data_request = requests.get('https://api3.binance.com/api/v3/klines?symbol=ETHBUSD&interval=5m&limit=1000')
# bnb_btc_data = data_request.json()
# data = pd.DataFrame(data = {
#     'open time': [],
#     'open': [],
#     'high': [],
#     'low': [],
#     'close': [],
# })

# for kline_item in bnb_btc_data :
#     time = kline_item[0] / 1000
#     row = {
#         'open time': datetime.utcfromtimestamp(time),
#         'open': float(kline_item[1]),
#         'high': float(kline_item[2]),
#         'low': float(kline_item[3]),
#         'close': float(kline_item[4]),
#     }
#     data = data.append(row, ignore_index=True)


# data.head()

# data.to_csv("working-data.csv", index=False)

data = pd.read_csv('ETHBUSD-5m.csv')

In [5]:
data['sma500'] = abstract.SMA(data, timeperiod=144)
data['LINEARREG'] = abstract.LINEARREG(data, timeperiod=144)
data['LINEARREG_ANGLE'] = abstract.LINEARREG_ANGLE(data, timeperiod=144)
data['LINEARREG_INTERCEPT'] = abstract.LINEARREG_INTERCEPT(data, timeperiod=144)
data['LINEARREG_SLOPE'] = abstract.LINEARREG_SLOPE(data, timeperiod=144)

macd = abstract.MACD(data, fastperiod=12, slowperiod=26, signalperiod=9)
data['macd'] = macd['macd']
data['macdsignal'] = macd['macdsignal']
data['macdhist'] = macd['macdhist']
data['rsi'] = abstract.RSI(data, timeperiod=24)
data.to_csv("data.csv", index=False)

In [4]:
money = 1000
sigPriceBuy = []
sigPriceSell = []

earns = pd.DataFrame(data = {
    'buy time': [],
    'buy': [],
    'sell': [],
    'sell time': [],
    'earn': [],
})

flag = 'sold'

for i in range(len(data)):
    if i <= 500:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue
        
    current_value = data.iloc[i]
    last_10 = data.iloc[i - 10: i]
        
    if flag == 'sold':
        is_higher_than_sma500 = current_value['close'] > current_value['sma500']

        is_positive_macdhist = current_value['macdhist'] > 0
        is_macd_up = current_value['macd'] > current_value['macdsignal']
        is_rsi_low = len(last_10[last_10['rsi'] < 30]) > 0

        buy_signal = is_positive_macdhist and is_macd_up and is_rsi_low
        
        if buy_signal:
            sigPriceBuy.append(data['close'][i])
            sigPriceSell.append(np.nan)
            flag = 'bought'
            earns = earns.append({ 
                'buy': data['close'][i],
                'buy time': data['open time'][i]
            }, ignore_index=True)
            continue

    elif flag == 'bought':
        is_negative_macdhist = current_value['macdhist'] < 0
        is_macd_down = current_value['macd'] < current_value['macdsignal']
        is_rsi_high = len(last_10[last_10['rsi'] > 70]) > 0
        
        sell_signal = is_negative_macdhist and is_macd_down and is_rsi_high
        
        rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
        
        buy_time = datetime.strptime(earns['buy time'][rowIndex], '%Y-%m-%d %H:%M:%S')
        
        current_time = datetime.strptime(current_value['open time'], '%Y-%m-%d %H:%M:%S')
        days_diff = (current_time - buy_time).total_seconds() / 86400
        money_diff = (current_value['close'] / earns['buy'][rowIndex]) - 1
        is_waiting_and_down = days_diff >= 1 and money_diff < 0.01
        
        
#         if sell_signal or money_diff <= -0.05:
        if (money_diff > 0.1) or money_diff < -0.05:
            rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(current_value['close'])
            flag = 'sold'
            earns.loc[rowIndex, 'sell'] = current_value['close']
            earns.loc[rowIndex, 'earn'] = current_value['close'] - earns.loc[rowIndex, 'buy'] 
            earns.loc[rowIndex, 'sell time'] = current_value['open time']
            continue
    else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue
   
        
earns.to_csv("earns.csv", index=False)        
print('Lucro', earns['earn'].sum())
total = earns['earn'].count()
print('Quantidade', earns['earn'].count())
wins = earns[earns['earn'] > 0].count()
p = (wins/total)['earn'] * 100
print(f'Porcentagem {p}%')
print('-------')
earns.to_csv("result.csv", index=False)

Lucro 717.2399999999997
Quantidade 100
Porcentagem 38.0%
-------
